In [1]:
import pandas        as pd
import numpy         as np

import matplotlib.pyplot as plt
import seaborn           as sns

In [2]:
clima = pd.read_parquet('/Users/aurelianosancho/Documents/Agro/clima_Agro.parquet', engine='pyarrow')
clima = clima.drop_duplicates(["Date"])

In [3]:
clima.head(3)

,Date,RegionID,dewpoint_temperature_2m,max_temperature_2m,min_temperature_2m,temperature_2m,humidity,surface_pressure,total_precipitation,u_component_of_wind_10m,v_component_of_wind_10m
0,2015-01-01,1100130,296.547072303263,304.2451299517846,296.6058847303008,299.3991562150645,84.34312620577637,99502.99053036416,0.05159416726036622,0.05000344746810352,-1.13460818997119
1,2015-01-01,1100205,296.75486460430403,303.4070397024061,296.4997753030794,299.03572449434046,87.26235160930503,99689.96080705665,0.0275239284449808,0.11688899210234413,-1.1804117958454605
2,2015-01-01,1100403,296.44428074847025,303.6416563826397,296.4401235938969,299.13535022267956,85.13912488000635,99500.52946351953,0.027933545470415863,0.30569423277731156,-1.3830371344155754
